In [ ]:
import pandas as pd
import numpy as np
import pymongo
import json
from pandas import read_csv, read_excel
import openpyxl


client = pymongo.MongoClient("mongodb://localhost:27017")
db=client.ameliaDB
summonerdata= db.summonerdata2 #summonerdata collection
matchdata=db.matchdata2 #matchdata collection
df=pd.DataFrame(columns=["blue_1_summoner_level","blue_2_summoner_level","blue_3_summoner_level","blue_4_summoner_level","blue_5_summoner_level","blue_kurtosis_summonerlevel","blue_coefficient_of_skewness_summonerlevel","blue_mean_summonerlevel","blue_variance_summonerlevel","blue_std_summonerlevel","blue_median_summonerlevel","blue_1_champ_points","blue_2_champ_points","blue_3_champ_points","blue_4_champ_points","blue_5_champ_points","blue_kurtosis_champpoints","blue_coefficient_of_skewness_champpoints","blue_mean_champpoints","blue_variance_champpoints","blue_std_champpoints","blue_median_champpoints","kills_1","kills_2","kills_3","kills_4","kills_5","damage1","damage2","damage3","damage4","damage5","assist1","assist2","assist3","assist4","assist5","red_1_summoner_level","red_2_summoner_level","red_3_summoner_level","red_4_summoner_level","red_5_summoner_level","red_kurtosis_summonerlevel","red_coefficient_of_skewness_summonerlevel","red_mean_summonerlevel","red_variance_summonerlevel","red_std_summonerlevel","red_median_summonerlevel","red_1_champ_points","red_2_champ_points","red_3_champ_points","red_4_champ_points","red_5_champ_points","red_kurtosis_champpoints","red_coefficient_of_skewness_champpoints","red_mean_champpoints","red_variance_champpoints","red_std_champpoints","red_median_champpoints","kills_1","kills_2","kills_3","kills_4","kills_5","damage1","damage2","damage3","damage4","damage5","assist1","assist2","assist3","assist4","assist5","blue_team_won?"])
print(df)


In [ ]:
namelist= matchdata.find({},{ "matchid":1,"match_data.info.participants":1,"_id":0})
name_data=[]
name_list=[]
summoner_level_list=[]
championid_list=[]
matchidlist=[]
blue_won=[]
kills=[]
damage=[]
assists=[]
for x in namelist:
    name_data.append(x['match_data']['info']['participants'])
    matchidlist.append(x['matchid'])
    



#print((name_data[0][0]),len(matchidlist))
for x in name_data:    
    blue_won.append(x[0]['win'])
    for k in range(0,10,1):
        name_list.append(x[k]['summonerName'])
        summoner_level_list.append(x[k]["summonerLevel"])
        championid_list.append(x[k]['championId'])
        kills.append(x[k]['kills'])
        damage.append(x[k]['totalDamageDealtToChampions'])
        assists.append(x[k]['assists'])
        

#print(championid_list)
print(len(matchidlist))
print(len(blue_won))                                                             

In [ ]:

def championmastery(name,champid):
    championpoints=0
    docs = summonerdata.aggregate([{ "$match": {"name":name, "champion_mastery": { "$elemMatch": { "$and": [{ "championId": champid }] } } } }, { "$project": { "champion_mastery": { "$filter": { "input": "$champion_mastery", "as": "champion_mastery", "cond": { "$and": [{ "$eq": ["$$champion_mastery.championId", champid] }] } } } } }])
    for doc in docs:
        championpoints=(doc['champion_mastery'][0]["championPoints"])        
        #print(championpoints)                    
    return championpoints    
        
def processmatchdata():   
    
    l,m,u=int(0),int(5),int(10)    
    for p in range(0,len(matchidlist)):
        print(len(matchidlist))
        blue_team_summonerlevel=[]
        red_team_summonerlevel=[]
        blue_team_masterypoints=[]
        red_team_masterypoints=[]
        blue_mastery_var=[]
        red_mastery_var=[]
        semi_final_blue=[]
        semi_final_red=[]
        final_row=[]
        bluestats_level=[]
        bluestats_mastery=[]
        redstats_level=[]
        redstats_mastery=[]
        blue_kills=[]
        blue_assists=[]
        blue_damage=[]
        red_kills=[]
        red_assists=[]
        red_damage=[]
        for y in range(int(l),int(m),1): #blue team
            blue_team_masterypoints.append(championmastery(name_list[y],championid_list[y]))
            blue_team_summonerlevel.append(summoner_level_list[y])
            sr_mastery_blue= pd.Series(blue_team_masterypoints)  
            sr_level_blue=pd.Series(blue_team_summonerlevel)
            blue_kills.append(kills[y])
            blue_damage.append(damage[y])
            blue_assists.append(assists[y])

            
            blue_var_mastery=sr_mastery_blue.var()    
            blue_var_level=sr_level_blue.var()     
            blue_std_mastery=sr_mastery_blue.std()    
            blue_std_level=sr_level_blue.std() 
            blue_mean_mastery=sr_mastery_blue.mean()    
            blue_mean_level=sr_level_blue.mean()  
            blue_median_mastery=sr_mastery_blue.std()    
            blue_median_level=sr_level_blue.median()   
            blue_kurtosis_mastery=sr_mastery_blue.kurtosis()    
            blue_kurtosis_level=sr_level_blue.kurtosis()
            blue_skewness_mastery=sr_mastery_blue.skew()    
            blue_skewness_level=sr_level_blue.skew()
            bluestats_level=[blue_kurtosis_level,blue_skewness_level,blue_mean_level,blue_var_level,blue_std_level,blue_median_level]
            bluestats_mastery=[blue_kurtosis_mastery,blue_skewness_mastery,blue_mean_mastery,blue_var_mastery,blue_std_mastery,blue_median_mastery]
            semi_final_blue = blue_team_summonerlevel+bluestats_level+blue_team_masterypoints+bluestats_mastery+blue_kills+blue_damage+blue_assists


        for j in range(m,u,1): #RED TEAM
            red_team_masterypoints.append(championmastery(name_list[j],championid_list[j]))
            red_team_summonerlevel.append(summoner_level_list[j])
            
            red_kills.append(kills[j])
            red_damage.append(damage[j])
            red_assists.append(assists[j])
            
            sr_mastery_red= pd.Series(red_team_masterypoints)  
            sr_level_red=pd.Series(red_team_summonerlevel)
            red_var_mastery=sr_mastery_red.var()    
            red_var_level=sr_level_red.var()
            red_std_mastery=sr_mastery_red.std()    
            red_std_level=sr_level_red.std() 
            red_mean_mastery=sr_mastery_red.mean()    
            red_mean_level=sr_level_red.mean()  
            red_median_mastery=sr_mastery_red.std()    
            red_median_level=sr_level_red.median()   
            red_kurtosis_mastery=sr_mastery_red.kurtosis()    
            red_kurtosis_level=sr_level_red.kurtosis()
            red_skewness_mastery=sr_mastery_red.skew()    
            red_skewness_level=sr_level_red.skew()
            redstats_level=[red_kurtosis_level,red_skewness_level,red_mean_level,red_var_level,red_std_level,red_median_level]
            redstats_mastery=[red_kurtosis_mastery,red_skewness_mastery,red_mean_mastery,red_var_mastery,red_std_mastery,red_median_mastery]
            semi_final_red = red_team_summonerlevel+redstats_level+red_team_masterypoints+redstats_mastery+red_kills+red_damage+red_assists
        l,m,u=l+10,m+10,u+10
        print("done ",p)
        final_row=(semi_final_blue+semi_final_red)
        final_row.append(int(blue_won[p]))
        
        print(final_row)
        print(len(final_row)) 
        df.loc[len(df)] = final_row   
        # blue_team_summonerlevel=[]
        # red_team_summonerlevel=[]
        # blue_team_masterypoints=[]
        # red_team_masterypoints=[] 
        
            
processmatchdata()
#df.to_csv("dataframe_second.csv")
df.to_excel("dataframe3.xlsx")
print(df)
    


In [52]:


train_df=read_csv("excel_comma.csv")
test_df=read_csv("excel2_comma.csv")
train_df=train_df.drop(columns=['Unnamed: 0.1','Unnamed: 0'])
test_df=test_df.drop(columns=['Unnamed: 0.1','Unnamed: 0'])
print(train_df.head())
print(test_df.head())
# #csvfile.to_csv("final_csv_Data")

   blue_1_summoner_level  blue_2_summoner_level  blue_3_summoner_level  \
0                    292                    360                    131   
1                     76                     39                    151   
2                    114                    116                    165   
3                     46                    263                    151   
4                    117                    155                    110   

   blue_4_summoner_level  blue_5_summoner_level  blue_kurtosis_summonerlevel  \
0                     48                     77                    -2.343819   
1                     76                     71                     3.036102   
2                    114                     95                     3.403980   
3                    136                    116                     1.822210   
4                    310                     29                     2.200739   

   blue_coefficient_of_skewness_summonerlevel  blue_mean_summonerlevel  \
